In [43]:
import os
import openai
import requests 
import tiktoken

In [2]:
from openai import OpenAI

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "is it too late to try the course?"}
    ])

In [12]:
response.choices[0].message.content


"It's never too late to start a new course or to continue your education. The best time to start is now! Here are a few things to consider:\n\n1. **Check the Enrollment Period**: Make sure you are still within the enrollment deadlines. Some courses may have specific start dates and deadlines, while others may offer rolling admission.\n \n2. **Prerequisites and Preparation**: Ensure you meet any prerequisites or requisite background knowledge for the course. If you need a refresher, consider reviewing the material beforehand.\n\n3. **Time Commitment**: Assess your current schedule to determine if you can dedicate the necessary time for coursework, assignments, and study.\n\n4. **Resources**: Gather all the necessary materials, such as textbooks, software, and lecture notes, before you start.\n\n5. **Support System**: Reach out to the course instructor or advisor if you have any questions or need guidance on how to catch up if you're starting late.\n\nRemember, lifelong learning is valua

## Q1. Getting the Data

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the Data

In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-homework"

es_client.indices.create(
    index=index_name,
    body=index_settings
)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework'})

In [10]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/mnt/e/llm-zoompcamp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:08<00:00, 114.80it/s]


## Q3. Searching

In [18]:
query = 'How do I execute a command in a running docker container?'

In [22]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
                }
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    # Extract the _score of the top-ranking result
    top_score = response['hits']['hits'][0]['_score'] if response['hits']['hits'] else None

    results_doc = []

    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])

    return top_score, results_doc



In [23]:
elastic_search(query)

(84.050095,
 [{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: Postg

## Q4. Filtering

In [24]:
query = 'How do I execute a command in a running docker container?'

In [25]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
                }
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    # Extract the _score of the top-ranking result
    top_score = response['hits']['hits'][0]['_score'] if response['hits']['hits'] else None

    results_doc = []

    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])

    return top_score, results_doc



In [30]:
results_doc = elastic_search(query)
results_doc

(84.050095,
 [{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to do

## Q5. Building a prompt

In [41]:
def build_prompt(query, search_results):
    context_template = """
        Q: {question}
        A: {text}
        """.strip()

    context = ""
    
    for doc in search_results:
        if isinstance(doc, dict) and 'question' in doc and 'text' in doc:
            # context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
            context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    

    # for doc in search_results:
    #     context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
    
        QUESTION: {question}
    
        CONTEXT:
        {context}
        """.strip()

    prompt = prompt_template.format(question=query, context=context.strip()).strip()

    return prompt


In [42]:
# Build the prompt
prompt = build_prompt(query, results_doc)

# Calculate the length of the resulting prompt
prompt_length = len(prompt)

print("Prompt Length:", prompt_length)

Prompt Length: 275


## Q6. Tokens

In [45]:
# Query
query = "How do I execute a command in a running docker container?"

# Build the prompt
prompt = build_prompt(query, results_doc)

# Initialize the encoding for the "gpt-4o" model
encoding = tiktoken.encoding_for_model("gpt-4o")

# Encode the prompt to get the number of tokens
tokens = encoding.encode(prompt)

# Calculate the number of tokens
num_tokens = len(tokens)

print("Number of Tokens:", num_tokens)

Number of Tokens: 54
